In [1]:
import os
import mne.io as io_eeg
import matplotlib.pyplot as plt
from TFG_utils import Load_Eeg_Info
from TFG_utils import Get_Frequency
from TFG_utils import Get_Start_End
from TFG_utils import spot_samples
import numpy as np
from scipy import signal
from sklearn.decomposition import PCA
import scipy
import statistics as stats
    
EEG_carpetas = []
EEG_Archivos = []
EEG_eeg = []
EEG_vhdr = []
EEG_vmrk = []

AWI_Normal = []      
AWI_Ad1 = []
AWI_Ad2 = []
AWI_Ad3 = []
AWI_Ad4 = []
AWI_Ad5 = []
AWI_Ad6 = []
    
def devolverCarpetas(carpeta):
    for archivo in os.listdir(carpeta):
        EEG_carpetas.append(os.path.join(carpeta,archivo))  
        
def devolverArchivos(carpeta):
    for archivo in os.listdir(carpeta):
        EEG_Archivos.append(os.path.join(carpeta,archivo))  

#######  Separamos cada archivo y creamos arrays.
devolverCarpetas("eeg_muestras")

i=0
while i < len(EEG_carpetas):
    devolverArchivos(EEG_carpetas[i])
    i += 1
    
for file in EEG_Archivos:
    (nombreFichero, extension) = os.path.splitext(file)
    if(extension == ".vhdr"):
        EEG_vhdr.append(nombreFichero+extension)
    elif(extension == ".vmrk"):
        EEG_vmrk.append(nombreFichero+extension)
    elif(extension == ".eeg"):
        EEG_eeg.append(nombreFichero+extension)

######  Comienza el procesado

n_muestra = 0
while n_muestra < len(EEG_vhdr):
    
    filename_vhdr = EEG_vhdr[n_muestra]
    eeg_object = io_eeg.read_raw_brainvision(filename_vhdr)
    
    t = eeg_object.times
    eeg = eeg_object.get_data()
    info = eeg_object.info
    
    # Load Eeg info in a new file called "Data_Info_Eeg"
    Eeg_Data = Load_Eeg_Info(info)
    
    # Get frequency
    fs = Get_Frequency(Eeg_Data)

    # Get start and end point in seconds of the whole eeg signal
    start_end = Get_Start_End(EEG_vmrk[n_muestra])
    
    # Get start point of each advice in seconds
    spots_times_sec = [0, 60, 120, 180, 226, 287, 347]
    spot_samples_eeg = spot_samples(start_end , fs, spots_times_sec)
    
    # Design and apply the "Notch" filter
    i=0
    eeg_channels_bp_array = [] 
    while i < 31:      #number of channels

        f0 = 50  # Frequency to be removed from signal (Hz)
        Q = 30.0  # Quality factor
        b,a = signal.iirnotch(f0,Q,int(fs))
        eeg_ch_notch = signal.filtfilt(b, a, eeg[i,0:len(t)])

        f1= 2
        f2= 30
        numtaps=64
        b=signal.firwin(numtaps, [f1, f2], pass_zero=False,fs=int(fs))
        eeg_chi_bp = signal.filtfilt(b, 1, eeg_ch_notch)

        eeg_channel = eeg_chi_bp # for instance
        eeg_channels_bp_array.append(eeg_channel)
        i += 1
    
    np_channels_bp_array = np.array(eeg_channels_bp_array)
    
    # Design and apply PCA filter
    pca = PCA(n_components=4)      
    principalComponents = pca.fit(np_channels_bp_array)
    components = pca.transform(np_channels_bp_array)
    filtered_channels_array = pca.inverse_transform(components)

    # Channels of interest [F3, F7, F4, F8]
    eeg_interest_channels = np.array([filtered_channels_array[2], 
        filtered_channels_array[3], filtered_channels_array[28], 
        filtered_channels_array[29]])

    # Each adviced and the part before the reproduction in one
    # array --> total: 7 Arrays 
    Prev = []
    Anuncio1 = []
    Anuncio2 = []
    Anuncio3 = []
    Anuncio4 = []
    Anuncio5 = []
    Anuncio6 = []

    j = 0
    while j < 4:
        i = 0
        while i < (len(spot_samples_eeg)-1):    
            x=np.arange(spot_samples_eeg[i],spot_samples_eeg[i+1])
            if i == 0:
                Prev.append(eeg_interest_channels[j, 
                    0:spot_samples_eeg[i]])
                Anuncio1.append(eeg_interest_channels[j, 
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            elif i == 1:
                Anuncio2.append(eeg_interest_channels[j,
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            elif i == 2:
                Anuncio3.append(eeg_interest_channels[j, 
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            elif i == 3:
                Anuncio4.append(eeg_interest_channels[j, 
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            elif i == 4:
                Anuncio5.append(eeg_interest_channels[j,
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            elif i == 5:
                Anuncio6.append(eeg_interest_channels[j,
                    spot_samples_eeg[i]:spot_samples_eeg[i+1]])
            i += 1
        j += 1
    
    np_prev = np.array(Prev)
    np_anuncio1 = np.array(Anuncio1)
    np_anuncio2 = np.array(Anuncio2)
    np_anuncio3 = np.array(Anuncio3)
    np_anuncio4 = np.array(Anuncio4)
    np_anuncio5 = np.array(Anuncio5)
    np_anuncio6 = np.array(Anuncio6) 

        ## Array = [A_prev , A_A1, A_A2, A_A3, A_A4, A_A5, A_A6] ##
    np_anuncios_t = [np_prev, np_anuncio1, np_anuncio2, 
                     np_anuncio3, np_anuncio4, np_anuncio5, np_anuncio6]
 
    # Estimated Powers
    
    pe_prev = []
    pe_anuncio1 = []
    pe_anuncio2 = []
    pe_anuncio3 = []
    pe_anuncio4 = []
    pe_anuncio5 = []
    pe_anuncio6 = []

    
###### Extracción de características

    # Apply Welch to each channel and advice
    j=0
    while j < 7:   ## anuncios + prev
        i = 0
        while i < 4:    # canales
            x = np_anuncios_t[j][i, 0:spot_samples_eeg[0]]
            f2, Pxx3 =scipy.signal.welch(x, fs = int(fs),
                        window='hamming', nperseg=128, nfft=1024)
            
            ## Select the alpha frequencies
            l=0
            alpha_frec = []
            while l < (len(f2)):
                if f2[l] >= 8:
                    if f2[l] <= 12: 
                        alpha_frec.append(l)  
                l += 1
                
            # calculo de la potencia estimada relativa        
            sumatorio_pxx3 = 0
            for k in Pxx3:
                sumatorio_pxx3 += k
                
            pxx3_rel = []
            for r in Pxx3:
                pxx3_rel.append(r/sumatorio_pxx3) 
            
            potencia_estimada = 0
            for x in alpha_frec:
                 potencia_estimada += pxx3_rel[x]
            
            
            potencia_estimada_rel = potencia_estimada
                
            # Arrays con cada una de las potencias 
            if j == 0:
                pe_prev.append(potencia_estimada_rel)
            elif j == 1:
                pe_anuncio1.append(potencia_estimada_rel)
            elif j == 2:
                pe_anuncio2.append(potencia_estimada_rel)
            elif j ==3:
                pe_anuncio3.append(potencia_estimada_rel)
            elif j == 4:
                pe_anuncio4.append(potencia_estimada_rel)
            elif j == 5:
                pe_anuncio5.append(potencia_estimada_rel)
            elif j ==6:
                pe_anuncio6.append(potencia_estimada_rel)
        
            i += 1
        j += 1

    # Potencias estimadas organizadas por canal 
    labels = ["Actividad normal ", "Anuncio 1", "Anuncio 2", "Anuncio 3",
              "Anuncio 4", "Anuncio 5", "Anuncio 6"]
    pptencias_estimadas_f3 = [pe_prev[0], pe_anuncio1[0], pe_anuncio2[0],
                pe_anuncio3[0], pe_anuncio4[0], pe_anuncio5[0], pe_anuncio6[0]]
    potencias_estimadas_f7 = [pe_prev[1], pe_anuncio1[1], pe_anuncio2[1],
                pe_anuncio3[1], pe_anuncio4[1], pe_anuncio5[1], pe_anuncio6[1]]
    potencias_estimadas_f4 = [pe_prev[2], pe_anuncio1[2], pe_anuncio2[2],
                pe_anuncio3[2], pe_anuncio4[2], pe_anuncio5[2], pe_anuncio6[2]]
    potencias_estimadas_f8 = [pe_prev[3], pe_anuncio1[3], pe_anuncio2[3],
                pe_anuncio3[3], pe_anuncio4[3], pe_anuncio5[3], pe_anuncio6[3]]
    
    # A. Withdrwal index 
        ## AW = GFPa_right - GFPa_left

    GFPa_right_prev = (potencias_estimadas_f3[0]+potencias_estimadas_f7[0])/2
    GFPa_left_prev = (potencias_estimadas_f4[0]+potencias_estimadas_f8[0])/2

    GFPa_right_anuncio1 = (potencias_estimadas_f3[1]+potencias_estimadas_f7[1])/2
    GFPa_left_anuncio1 = (potencias_estimadas_f4[1]+potencias_estimadas_f8[1])/2

    GFPa_right_anuncio2 = (potencias_estimadas_f3[2]+potencias_estimadas_f7[2])/2
    GFPa_left_anuncio2 = (potencias_estimadas_f4[2]+potencias_estimadas_f8[2])/2

    GFPa_right_anuncio3 = (potencias_estimadas_f3[3]+potencias_estimadas_f7[3])/2
    GFPa_left_anuncio3 = (potencias_estimadas_f4[3]+potencias_estimadas_f8[3])/2

    GFPa_right_anuncio4 = (potencias_estimadas_f3[4]+potencias_estimadas_f7[4])/2
    GFPa_left_anuncio4= (potencias_estimadas_f4[4]+potencias_estimadas_f8[4])/2

    GFPa_right_anuncio5 = (potencias_estimadas_f3[5]+potencias_estimadas_f7[5])/2
    GFPa_left_anuncio5 = (potencias_estimadas_f4[5]+potencias_estimadas_f8[5])/2

    GFPa_right_anuncio6 = (potencias_estimadas_f3[6]+potencias_estimadas_f7[6])/2
    GFPa_left_anuncio6 = (potencias_estimadas_f4[6]+potencias_estimadas_f8[6])/2

    AW_prev = GFPa_right_prev - GFPa_left_prev
    AWI_Normal.append(AW_prev)
    AW_anuncio1 = GFPa_right_anuncio1 - GFPa_left_anuncio1
    AWI_Ad1.append(AW_anuncio1)
    AW_anuncio2 = GFPa_right_anuncio2 - GFPa_left_anuncio2
    AWI_Ad2.append(AW_anuncio2)
    AW_anuncio3 = GFPa_right_anuncio3 - GFPa_left_anuncio3
    AWI_Ad3.append(AW_anuncio3)
    AW_anuncio4 = GFPa_right_anuncio4 - GFPa_left_anuncio4
    AWI_Ad4.append(AW_anuncio4)
    AW_anuncio5 = GFPa_right_anuncio5 - GFPa_left_anuncio5
    AWI_Ad5.append(AW_anuncio5)
    AW_anuncio6 = GFPa_right_anuncio6 - GFPa_left_anuncio6
    AWI_Ad6.append(AW_anuncio6)
    
    n_muestra += 1

###### Representación Gráfica poblacional
    
mean_AWI = [stats.mean(AWI_Normal), stats.mean(AWI_Ad1), 
            stats.mean(AWI_Ad2), stats.mean(AWI_Ad3), 
            stats.mean(AWI_Ad4),stats.mean(AWI_Ad5), stats.mean(AWI_Ad6)]
desv_AWI = [np.std(AWI_Normal), np.std(AWI_Ad1), np.std(AWI_Ad2),
            np.std(AWI_Ad3), np.std(AWI_Ad4), np.std(AWI_Ad5),
            np.std(AWI_Ad6)]

# Define labels, positions, bar heights and error bar heights
labels = ['Basal', 'Anuncio 1', 'Anuncio 2', 
          'Anuncio 3', 'Anuncio 4', 'Anuncio 5', 'Anuncio 6']
x_pos = np.arange(len(labels))
CTEs = mean_AWI
error = desv_AWI

fig, ax = plt.subplots(figsize=(8, 5))
ax.errorbar(x_pos, CTEs, yerr = error, fmt = "o ", capsize= 10)
ax.set_ylabel('AW index')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
plt.plot()
    

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'eeg_muestras'